# Formularios y validadores
Web2py define cuatro ayudantes para crear formularios HTML
- FORM: Ayudante HTML que genera un formulario
- SQLFORM: Como FORM, pero con acceso a la base de datos
- SQLFORM.factory: Simulación de SQLFORM
- CRUD: Sustituido por SQLFORM.grid y SQLFORM.smartgrid

Todos ellos tienen la capacidad de observarse, validarse y modificarse en función de la validación y los errores derivados de ella.

Dado que FORM y SQLFORM son ayudantes HTML, es posible modificar su contenido.
```python
form = SQLFORM(..)
form['_style']='border:1px solid black'
```

In [1]:
import sys
from IPython.display import display, Code as html_code


web2py_path = '../../curso/web2py'
sys.path.append(web2py_path)

# ayudantes HTML de Web2py
from gluon.html import *
# función de Jupyter que renderiza HTML
from IPython.core.display import HTML as IP_HTML

# función de apoyo para renderizar el HTML
# generado por Web2py
def w2p_render(html, verbose=False):
    'html -> (html, html_renderizado)'
    if verbose:
        display(html_code(str(html)))
    if 'Rows' in str(html.__class__):
        html = SQLTABLE(html)
    return IP_HTML(str(html))

## [FORM](http://www.web2py.com/books/default/chapter/29/07/forms-and-validators#FORM)
Veamos un ejemplo:

Acción:
```python
def formulario():
    # <form>Nombre:<input name="nombre"><input type="submit"></form>
    form = FORM('Nombre:', INPUT(_name='nombre'), INPUT(_type='submit'))
    return dict(form=form) 
```

Vista:
```html
{{extend 'layout.html'}}
<h2>Formulario</h2>
<form enctype="multipart/form-data"
      action="{{=URL()}}" method="post">
Nombre:
<input name="nombre" />
<input type="submit" />
</form>
<h2>Variables enviadas</h2>
{{=BEAUTIFY(request.vars)}}
```

El formulario pide un nombre y lo envía (submit) a la URL definida. En este caso action=URL(), lo que indica que se enviará de vuelta a la acción que lo generó.

Dicha acción recibirá el formulario y su contenido estará disponible en request.vars. En este caso request.vars.nombre, y volverá a emitir un formulario vacío.

La vista mostrará ahora el formulario vacío y al pie request.vars.

Lo mismo con ayudantes:
```python
def formulario():
   form = FORM('Nombre:', INPUT(_name='nombre'), INPUT(_type='submit'))
   return dict(form=form)
```
```html
{{extend 'layout.html'}}
<h2>Formulario</h2>
{{=form}}
<h2>Variables enviadas</h2>
{{=BEAUTIFY(request.vars)}}
```

Ahora es el propio ayudante HTML form, el que se renderiza en la vista.

Podemos añadir validación y procesado:
```python
def formulario():
    form = FORM('Nombre:',
                INPUT(_name='nombre', requires=IS_NOT_EMPTY()),
                INPUT(_type='submit'))
    if form.accepts(request, session):
        response.flash = 'Aceptado'
    elif form.errors:
        response.flash = 'El formulario tiene errores'
    else:
        response.flash = 'Por favor, escribe tu nombre'
    return dict(form=form)
```
```html
{{extend 'layout.html'}}
<h2>Formulario</h2>
{{=form}}
<h2>Variables enviadas</h2>
{{=BEAUTIFY(request.vars)}}
<h2>Variables aceptada</h2>
{{=BEAUTIFY(form.vars)}}
<h2>Errores en el formulario</h2>
{{=BEAUTIFY(form.errors)}}
```
Avisos:
- requires=IS_NOT_EMPTY() es un validador del input nombre
- En la acción comprobamos la recepción con form.accepts(...)
- En la vista mostramos form.vars, form.errors y request.vars

Todo el trabajo lo hace el objeto form.
- Recibe request.vars
- Valida los requisitos definidos por los validadores
- Guarda las variables aceptadas en form.vars
- Guarda los errores de validación en form.errors
- Realiza tareas de BB.DD.

### Firma completa de form.accepts
```python
form.accepts(vars,          # request.vars, request.get_vars, request.post_vars o request
             session=None,
             formname='default',
             keepvalues=False,
             onvalidation=None,
             dbio=True,
             hideerror=False):
```
El método accepts devuelve True si el formulario ha sido procesado sin errores y False en caso contrario.

Un atajo de form.accepts(...) es form.process().accepted. Resulta más cómodo, porque no es necesario pasar a proccess todos los argumentos de accepts.

El método process tiene además otros argumentos
- message_onsuccess: mensaje en caso de éxito
- onsuccess: si es 'flash' (lo es por defecto) y el formulario es aceptado mostrará el flash message_onsuccess
- message_onfailure: mensaje en caso de fallo
- onfailure: como onsuccess
- next: URL a la que será redirigida la respuesta si el formulario es aceptado

onsuccess y onfailure pueden ser una función/lambda (lambda form: haz_algo_con_el_formulario(form))

form.validate(...) es un atajo de form.process(..., dbio=False).accepted

In [2]:
form = FORM('Nombre:',
            INPUT(_name='nombre'),
            INPUT(_type='submit'))
w2p_render(form, verbose=True)

<form action="#" enctype="multipart/form-data" method="post">Nombre:<input name="nombre" type="text" /><input type="submit" /></form>

### Campos ocultos
Si observamos el código fuente generado por un formulario veremos que hay un par de campos ocultos:
- _formkey: evita el doble envío de formularios
- _formname: útil en caso de páginas con varios formularios
```html
<form enctype="multipart/form-data" action="" method="post">
your name:
<input name="nombre" />
<input type="submit" />
<input value="783531473471" type="hidden" name="_formkey" />
<input value="default" type="hidden" name="_formname" />
</form>
```
Si se produce algún error en el procesado del formulario:
```html
<form enctype="multipart/form-data" action="" method="post">
your name:
<input value="" name="nombre" />
<div class="error">cannot be empty!</div>
<input type="submit" />
<input value="783531473471" type="hidden" name="_formkey" />
<input value="default" type="hidden" name="_formname" />
</form>
```
Web2py inserta un nuevo div con la class "error" en el formulario, para informar del error.

La plantilla "layout.html" maneja los divs con clase "error". Por defecto utiliza efectos jQuery para hacer que los errores aparezcan y desaparezcan en rojo.

### keepvalues
Indica a Web2py que cuando un formulario sea aceptado, deje el contenido de los campos en lugar de vaciarlo.

### dbio
Si es False, Web2py no realizará tareas sobre la base de datos.

### hideerror
Si es True y el formulario tiene errores, estos no se mostrarán.

### onvalidation
Puede ser None o una función que recibe el formulario y no devuelve nada. Esta función se llamará después de la validación (si la ha pasado) y antes de cualquier otra cosa. Puede utilizarse para varias cosas:
- añadir chequeos adicionales y añadir errores al formulario
- calcular el valor de algún campo en función de otros
- lanzar alguna acción antes de que el registro sea creado o modificado, por ejemplo enviar un correo

### Detectar el cambio de un registro
Es posible que varios usuarios traten de modificar el mismo registro de modo simultáneo. Esto únicamente tiene sentido en SQLFORM.
```python
def edita_registro():
    reg = db.registro(request.args(0)) or redirect(URL('error'))
    form=SQLFORM(db.registro, reg)
    form.process(detect_record_change=True)
    if form.record_changed:
        # lo que sea
    elif form.accepted:
        # lo que sea
    else:
        # lo que sea
    return dict(form=form)
```

### Formularios y redirección
Dado que lo habitual es que los formularios se envíen a la misma acción que los genera, cuando el formulario es aceptado tendremos que hacer algo para no entrar en bucle.
```python
def formulario():
    form = FORM('Nombre:',
              INPUT(_name='nombre', requires=IS_NOT_EMPTY()),
              INPUT(_type='submit'))
    if form.process().accepted:
        session.flash = 'Ok'
        redirect(URL('next'))
    elif form.errors:
        response.flash = 'Error'
    else:
        response.flash = 'Rellena el formulario'
    return dict(form=form)

def next():
    return dict()
```

### Múltiples formularios por página
SQLFORM ya da nombres distintos a formularios de distintas tablas, pero FORM no hace lo mismo.
```python
def dos_formularios():
    form1 = FORM(INPUT(_name='nombre', requires=IS_NOT_EMPTY()),
                 INPUT(_type='submit'), _name='form_uno')
    form2 = FORM(INPUT(_name='nombre', requires=IS_NOT_EMPTY()),
                 INPUT(_type='submit'), _name='form_dos')
    if form1.process(formname='form_uno').accepted:
        response.flash = 'form uno aceptado'
    if form2.process(formname='form_dos').accepted:
        response.flash = 'form dos aceptado'
    return dict(form1=form1, form2=form2)
```

### Compartiendo formularios
```python
form = FORM(INPUT(_name='nombre', requires=IS_NOT_EMPTY()),
        INPUT(_type='submit'), _action=URL('pagina_dos'))

def pagina_uno():
    return dict(form=form)

def pagina_dos():
    if form.process(session=None, formname=None).accepted:
         response.flash = 'Ok'
    else:
         response.flash = 'Error'
    return dict()
```

### Añadir botones a FORMs
```python
form.add_button('Back', URL('other_page'))
```

### Manipulación de los FORMs
Ya hemos visto en el tema de las vistas que FORM es un ayudante HTML, lo que permite que se manipule accediendo a sus componentes.

## SQLFORM
Supongamos:
```python
# modelo
db = DAL('sqlite://storage.sqlite')
db.define_table('persona', Field('nombre', requires=IS_NOT_EMPTY()))

# controlador
def formulario():
   form = SQLFORM(db.persona)
   if form.process().accepted:
       response.flash = 'Ok'
   elif form.errors:
       response.flash = 'Error'
   else:
       response.flash = 'Rellena el formulario'
   return dict(form=form)
```
```html
<!-- vista -->
{{extend 'layout.html'}}
<h2>Formulario</h2>
{{=form}}
```

Ahora el formulario se ha construido mediante SQLFORM que recibe como parámetro el nombre de una tabla, de modo que es capaz de construir un formulario partiendo del modelo definido para dicha tabla. Se renderizaría así:
```html
<form enctype="multipart/form-data" action="" method="post">
  <table>
    <tr id="persona_nombre__row">
       <td><label id="persona_nombre__label"
                  for="person_nombre">Nombre: </label></td>
       <td><input type="text" class="string"
                  name="name" value="" id="persona_nombre" /></td>
       <td></td>
    </tr>
    <tr id="submit_record__row">
       <td></td>
       <td><input value="Submit" type="submit" /></td>
       <td></td>
    </tr>
  </table>
  <input value="9038845529" type="hidden" name="_formkey" />
  <input value="persona" type="hidden" name="_formname" />
</form>
```
Se puede observar como el formulario HTML generado por Web2py es más complejo que el que genera para FORM.
- Está contenido en una tabla
    - La primera columna contiene las etiquetas de los campos
    - La segunda columna contiene los campos y los errores si los hubiera
    - La tercera está vacía
- Los elementos HTML contienen atributos que se refieren a la tabla origen del formulario y sus campos.
- Ahora, el método accepts() hace más cosas
    - Validación
    - Modificaciones en la base de datos
    - Almacena en form.var.id el id del registro si se trata de un alta
    - Gestiona la carga y almacenamiento de registros con campos de tipo "upload"
        - Los guarda en el directorio oportuno tras nombrarlos de forma segura y guarda su nombre en la tabla correspondiente. Guarda en form.vars.fieldname, el nombre oportuno.
        - Atención: La longitud por defecto del campo nombre de estos fichero es 512. Se puede elegir otra longitud Field(..., length=...).

SQLFORM renderiza los campos de las tablas en base a su tipo:
- boolean: checkboxes
- text: textareas
- valores pertenecientes a un conjunto: drop-downs
- upload: enlaces al fichero
- blob: los oculta, ya que se supone que el programador ha de definir un tratamiento para ellos.

Es posible personalizar mucho un SQLFORM
- Mostrar un subconjunto de campos
- Cambiar la etiquetas
- Mostrar valores en la columna libre
- Crear formularios de actualización y borrado

La clase que define SQLFORM está en "gluon/sqlhtml.py" y se puede extender fácilmente, sobreescribiendo su método xml, ya que es el encargado de serializarlo.

### Firma de SQLFORM:
```python
SQLFORM(table,
        record=None,         # si se proporciona un registro, el formulario será para modificarlo
        deletable=False,     # si es True, se muestra un check que permite borrar el registro al ser enviado
        linkto=None,         # URL hacia registros referenciados
        upload=None,         # URL hacia registros referenciados
        fields=None,         # puede contener una lista de los campos que se desea mostrar
        labels=None,         # diccionario de las etiquetas de los campos {'name': 'Nombre', ...}
        col3={},             # diccionario de valores para la 3ª columna {'name': A('Enlace', _href='http...', ...}
        submit_button='Submit',           # texto a mostrar en el botón de envío
        delete_label='Check to delete',   # etiqueta del check para borrar registro
        showid=True,                      # muestra el id del registro?, id_label -> etiqueta del id
        readonly=False,      # ¿está claro?
        comments=True,       # muestra o no los comentarios
        keepopts=[],
        ignore_rw=False,     # Si True se ignoran las restricciones impuestas a los campos db.tabla.campo.writable/readable
        record_id=None,
        formstyle='table3cols',   # estilo del formulario, previamente definido
        buttons=['submit'],  # lista de inputs, buttons o ayudantes HTML que irán al lado del botón submit
        separator=': ',      # separa la etiqueta del campo
        extra_fields=None,   # campos extra a añadir
        **attributes)        # atributos que pasan a FORM, comienzan por "_"           
```

Hay un atributo especial, hidden, que puede contener un diccionario con los campos que deseamos que permanezcan ocultos en el formulario. Estos campos son enviados con el submit, pero form.accpets() no los lee ni los pasa a form.vars.

#### formstyle
Obtiene su valor de response.formstyle, pero hay unas cuantas opciones predefinidas:
- bootstrap4_inline: Este es el valor por defecto
- bootstrap4_stacked
- bootstrap3_inline
- bootstrap3_stacked
- bootstrap2
- table3cols
- table2cols
- ul
- divs: ideal para posteriores personalizaciones
- bootstrap: bootstrap 2.3 "form-horizontal"
- Además puede ser una función que genere todo el HTML deseado. Dicha función recibirá el formulario y los campos. Se puede ver el código en sqlhtml.py (busca las funciones llamadas "formstyle_")


### El método process
SQLFORM hereda este método de FORM.

### SQLFORM y HTML
En ocasiones usamos SQLFORM para aprovechar su conocimiento del modelo y su procesado, pero necesitamos personalizar su aspecto y no lo podemos conseguir mediante sus parámetros. En esos casos, lo escribimos por completo en HTML.
```python
def formulario_manual():
    form = SQLFORM(db.persona)
    if form.process(session=None, formname='test').accepted:
        response.flash = 'Ok'
    elif form.errors:
        response.flash = 'Error'
    else:
        response.flash = 'Rellena el formulario'
    # No envío nada a la vista
    return dict()
```
```html
<!-- formulario_manual.html -->
{{extend 'layout.html'}}
<form action="#" enctype="multipart/form-data" method="post">
<ul>
  <li>Nombre: <input name="nombre" /></li>
</ul>
  <input type="submit" />
  <input type="hidden" name="_formname" value="test" />
</form>
<!-- el formulario se crea totalmente en la vista
y se relaciona con el creado en la acción por el atributo
formname del método process
en la vista hemos añadido un input oculto con el nombre
y el valor oportunos -->
```

En este caso, en el que la vista contiene un único formulario, no habría sido necesario forzar un formname, ya que Web2py genera uno automáticamente. Pero hay que tenerlo en cuenta para vistas con múltiples formularios.

form.accepts busca en response.vars aquellos campos que coinciden con los de la tabla db.persona.

**Importante: El formulario HTML tiene que utilizar el método POST y el encoding multipart/form-data.**

### SQLFORM y uploads
Los campos de tipo "upload" son especiales.
- Se renderizan como inputs de tipo file
- Por defecto se hace streaming utilizando un buffer y se guardan en el directorio "uploads" de la aplicación usando un nuevo nombre asignado automáticamente
- Ese nombre es lo que se guarda en el campo de tipo upload de la tabla
```python
db.define_table('foto',
    Field('nombre', requires=IS_NOT_EMPTY()),
    Field('imagen', 'upload'))
```

Al insertar un registro, se puede explorar en busca de un fichero y subirlo. Se guardará como applications/app/uploads/foto.imagen.XXXXX.jpg. Donde "XXXXXX" es un identificador aleatorio. Por defecto, el nombre original es codificado (b16encoded) y utilizado para construir el nuevo nombre del fichero. Este nombre se recupera por la acción por defecto "download" y usado en su descarga. Se preserva su extensión. Esto es así por seguridad, evitando que los caracteres especiales en el nombre de un fichero permitan realizar operaciones maliciosas.

El nuevo nombre del fichero se devuelve por form.vars.imagen.

Pasando una URL al SQLFORM constructor mediante su argumento "upload", Web2py usa dicha acción para descargar el fichero:
```python
def foto():
   record = db.foto(request.args(0))
   form = SQLFORM(db.foto,
                  record,
                  deletable=True,
                  upload=URL('descarga'))
   if form.process().accepted:
       response.flash = 'Ok'
   elif form.errors:
       response.flash = 'Error'
   return dict(form=form)

def descarga():
    return response.download(request, db)
```
Si insertamos una imagen y luego la visitamos, veremos un formulario cuyo HTML sería de la forma:
```html
<td><label id="foto_imagen__label" for="foto_imagen">Imagen: </label></td>
<td><div><input type="file" id="foto_imagen" class="upload" name="imagen"
/>[<a href="/app/default/download/foto.image.0246683463831.jpg">file</a>|
<input type="checkbox" name="imagen__delete" />delete]</div></td><td></td></tr>
<tr id="delete_record__row"><td><label id="delete_record__label" for="delete_record"
>Check to delete:</label></td><td><input type="checkbox" id="delete_record"
class="delete" name="delete_this_record" /></td>
```
Que contiene:
- Un enlace al fichero a descargar
- Un checkbox para borrar el registro de la base de datos

¿Por qué es necesario escribir la acción "descarga"? Porque es posible que se desee algún tipo de autorización.

### Guardar el nombre original del fichero
Si es necesario:
```python
db.define_table('foto',
    Field('persona', requires=IS_NOT_EMPTY()),
    Field('nombre_original'),
    Field('nombre_web2py', 'upload'))

def form_persona():
    record = db.persona(request.args(0)) or redirect(URL('index'))
    url = URL('descarga')
    form = SQLFORM(db.persona,
                   record,
                   deletable=True,
                   upload=url,
                   fields=['persona', 'nombre_web2y'])
    if request.vars.nombre_web2py != None:
        form.vars.nombre_original = request.vars.persona.nombre_web2py
    if form.process().accepted:
        response.flash = 'Ok'
    elif form.errors:
        response.flash = 'Error'
    return dict(form=form)
```
SQLFORM no muestra el campo nombre_original.

### autodelete
SQLFORM no borra automáticamente los ficheros cuando se borran los registros asociados. Si se quiere forzar, basta con informar en el campo correspondiente el parámetro "autodelete=True"

### Enlaces a registros relacionados
En el caso de tablas relacionadas, es posible que se quiera que en el formulario de un registro exista un enlace a los registros con los que se relacione.
```python
# modelo
db.define_table('persona',
    Field('nombre', requires=IS_NOT_EMPTY()))

db.define_table('mascota',
    Field('persona', 'reference persona'),
    Field('nombre', requires=IS_NOT_EMPTY()))

db.mascota.persona.requires = IS_IN_DB(db, 'persona.id', '%(nombre)s')

# acción
def persona():
   record = db.persona(request.args(0)) or redirect(URL('index'))
   url = URL('lista_registros', args='db')
   form = SQLFORM(db.persona, record, deletable=True, linkto=url)
   if form.process().accepted:
       response.flash = 'Ok'
   elif form.errors:
       response.flash = 'Error'
   return dict(form=form)
```

En la página resultante aparece un enlace "mascota.persona", que puede cambiarse mediate el argumento "labels" del SQLFORM.

El enlace llevará a /app/default/lista_registros/db/mascota?query=db.mascota.persona%3D%3D3

La acción "lista_registros recibe en request.args(0) el nombre de la tabla referenciada y en request.vars.query la consulta SQL "mascota.persona==3".
```python
def lista_registros():
    import re
    REGEX = re.compile(r'^(\w+).(\w+).(\w+)==(\d+)$')
    match = REGEX.match(request.vars.query)
    if not match:
        redirect(URL('error'))
    tabla, campo, id = match.group(2), match.group(3), match.group(4)
    records = db(db[tabla][campo]==id).select()
    return dict(records=records)
```
```html
<!-- vista list_records.html" -->
{{extend 'layout.html'}}
{{=records}}
```

Cuando una vista recibe un conjunto de registros, primero se convierten mediante el ayudante SQLTABLE y luego se serializan a HTML, poniendo cada campo en una columna y cada registro en una fila.

### Asignando valores a los campos del formulario
```python
# esto debe hacerse después de crear el form
# y antes de que sea aceptado
# independientemente de que el campo "nombre" se muestre
form.vars.nombre = 'valor'
```

### Añadiendo elementos extra al SQLFORM
```python
form = SQLFORM(db.tabla)
# al añadir "a mano" elementos extra debemos tener en cuenta
# el estilo del formulario
elemento_extra = TR(LABEL('Acepto las condiciones'),
                    INPUT(_name='acepto', value=True, _type='checkbox'))
form[0].insert(-1, elemento_extra)
```

### SQLFORM sin acceso a la base de datos
Es posible "simular" el comportamiento de SQLFORM, pero sin consecuencias sobre la base de datos. Por ejemplo, en los casos en los que el contenido de un campo deba computarse antes de reflejarse en su tabla correspondiente y ese cálculo sea tan complejo que no pueda llevarse a cabo con los tratamientos por defecto de Web2py. Otra posibilidad es el caso de una validación que se escape a los validadores Web2py.
```python
para ello modificaríamos el flujo normal:
form = SQLFORM(db.persona)
if form.process().accepted:
    response.flash = 'Ok'

# por
form = SQLFORM(db.persona)
if form.validate():
    ### gestionaríamos las cargas explícitamente
    # haz lo que debas
    form.vars.id = db.persona.insert(**dict(form.vars))
    response.flash = 'Ok'

# lo mismo sucede con los formularios para actualización y borrado
# sustituiríamos
form = SQLFORM(db.persona, registro)
if form.process().accepted:
    response.flash = 'Ok'

# por
form = SQLFORM(db.persona, registro)
if form.validate():
    if form.deleted:
        db(db.persona.id==registro.id).delete()
    else:
        form.record.update_record(**dict(form.vars))
    response.flash = 'Ok'
```

## SQLFORM.factory
Permite generar formularios como si existiera un modelo, aunque no sea así.
```python
def form_factory():
    form = SQLFORM.factory(
        Field('nombre', requires=IS_NOT_EMPTY()),
        Field('foto', 'upload'))
    if form.process().accepted:
        response.flash = 'Ok'
        session.nombre = form.vars.nombre
        session.foto = form.vars.foto
    elif form.errors:
        response.flash = 'Error'
    return dict(form=form)
```
```html
<!-- default/form_factory.html -->
{{extend 'layout.html'}}
{{=form}}
```

No está permitido utilizar espacios en los nombres de los campos, ni pasar un diccionario de etiquetas, como con SQLFORM. Por defecto SQLFORM.factory crea el formulario utilizando el atributo HTML "id" como si el formulario hubiera sido creado desde una tabla llamada "no_table". Se puede cambiar:
```python
form = SQLFORM.factory(..., table_name='nombre_deseado')
```
Es útil para generar varios formularios de la misma tabla y evitar conflictos CSS.

### Un formulario para varias tablas
En ocasiones se desea mostrar un formulario para varias tablas relacionadas:
```python
# modelo
db.define_table('cliente',
    Field('nombre'))
db.define_table('direccion',
    Field('cliente', 'reference cliente', writable=False, readable=False),
    Field('calle'),
    Field('ciudad'))

# controlador
def nuevo_cliente():
    form = SQLFORM.factory(db.cliente, db.direccion)
    if form.process().accepted:
        # esto no funcionaría si las tablas tuvieran campos en común
        id = db.cliente.insert(**db.cliente._filter_fields(form.vars))
        form.vars.client = id
        id = db.direccion.insert(**db.direccion._filter_fields(form.vars))
        response.flash = 'Ok'
    return dict(form=form)
```

### Formularios personalizados
Para los formularios creados con SQLFORM, SQLFORM.factory o CRUD:
```python
db.define_table('imagen',
    Field('nombre', requires=IS_NOT_EMPTY()),
    Field('fuente', 'upload'))

# acción
def carga_imagen():
    return dict(form=SQLFORM(db.imagen).process())
```
```html
<!-- vista estándar -->
{{=form}}
```
``` html
<!-- vista con un formulario personalizado
es posible acceder a los componentes del formulario -->
{{=form.custom.begin}}
Nombre: <div>{{=form.custom.widget.nombre}}</div>
Fichero: <div>{{=form.custom.widget.fuente}}</div>
{{=form.custom.submit}}
{{=form.custom.end}}
<!-- 
form.custom.widget[campo] se serializa tal y como define el widget del campo
si se producen errores, se muestran correctamente
-->
```
Si no se desea utilizar los widgets definidos por Web2py, es posible sustituirlos por HTML crudo. En ese caso tenemos unos cuantos objetos que nos serán de utilidad:
```python
form.custom.label[campo]    # etiqueta del campo
form.custom.comment[campo]  # comentario del campo
form.custom.dspval[campo]   # representación del campo
form.custom.inpval[campo]   # representación del input
form.custom.delete          # si el formulario tiene deletable=True -> checkbox
```

### Convenio CSS
Es importante conocer las reglas que utiliza Web2py a la hora de crear formularios con SQLFORM, SQLFORM.factory y CRUD si queremos personalizarlos.

Para la tabla "mitabla", el campo "micampo" de tipo "string", se utilizará SQLFORM.widgets.string.widget, que serializará el campo:
```html
<input type="text" name="myfield" id="mytable_myfield" class="string" />
```
**Aviso**:
- La clase del input es la misma que el tipo de campo. El código JS definido en "web2py_ajax.html" necesita que sea así para funcionar
- Asegura el formato numérico para campos de tipo "integer" y "double"
- Utiliza un popup para los de tipo "time", "date" y "datetime"
- El id es nombre-tabla_nombre-campo. Esto permite referencias posteriores, como jQuery('#mytable_myfield') o document.getElementById('#mytable_myfield')
- El atributo name es el nombre del campo

### Ocultar los errores
En el caso de que se desee evitar el mecanismo por defecto para la visualización de los errores, basta con utilizar:
- Para FORM o SQLFORM: hideerror=True en el método accepts
- Para CRUD: crud.settings.hideerror=True

Habrá que modificar la vista para mostrar los errores de la forma deseada.
```html
<!-- este podría ser un ejemplo -->
{{if form.errors:}}
  Algo no está bien:
  <ul>
  {{for campo in form.errors:}}
    <li>{{=campo}} error: {{=form.errors[campo]}}</li>
  {{pass}}
  </ul>
  {{form.errors.clear()}}
{{pass}}
{{=form}}
```

## [Validadores](http://www.web2py.com/books/default/chapter/29/07/forms-and-validators#Validators)
Son clases definidas por Web2py para validar los inputs.
```python
# ejemplos de validadores
# en un objeto INPUT
input_a = INPUT(_name='a', requires=IS_INT_IN_RANGE(0, 10))

# en un campo de una tabla
db.define_table('persona', Field('nombre'))
db.persona.nombre.requires = IS_NOT_EMPTY()
```

Siempre se asignan mediante el atributo "requires" de un campo. Un campo puede tener uno o varios validadores. Se pueden utilizar listas de validadores.
```python
db.persona.nombre.requires = [IS_NOT_EMPTY(),
                              IS_NOT_IN_DB(db, 'persona.nombre')]
```

Se llaman automáticamente, en el orden en que fueron declarados, por los métodos "accepts" and "process" del formulario.

Pero se pueden llamar explícitamente:
```python
db.persona.nombre.validate(valor)   # devuelve la tupla (valor, error_o_None)
```

Los validadores tienen un argumento opcional:
```python
IS_NOT_EMPTY(error_message='no puede estar vacío')   # error_message sobreescribe el mensaje por defecto del validador
```

Los mensajes de error definidos por defecto no se traducen.

### Validadores para campos de tipo "list:"
- IS_IN_DB(..., multiple=True): multiple=(1, 1000) exige la elección entre 1 y 1000 items, al menos uno
- IS_IN_SET(..., multiple=True)
- IS_NOT_EMPTY()
- IS_LIST_OF_EMAILS()
- IS_LIST_OF(...): Puede utilizarse para aplicar cualquier validador a los items individuales de la lista

### Validadores para campos de tipo "text"
- IS_ALPHANUMERIC: a-z, A-Z, o 0-9
- IS_LOWER: minúsculas
- IS_UPPER: mayúsculas
- IS_EMAIL
- IS_MATCH: expresión regular, requires = IS_MATCH('^\d{1,3}(.\d{1,3}){3}$')
- IS_LENGTH(max, min): longitud
- IS_URL: RFC 2616[RFC2616]
- IS_SLUG: únicamente caracteres alfanuméricos y guiones no consecutivos
- IS_JSON

### Date and time validators
- IS_TIME: todos incluyen el formato, mediante el argumento "format"
- IS_DATE
- IS_DATETIME
- IS_DATE_IN_RANGE
- IS_DATETIME_IN_RANGE

Variables posibles a incluir en la cadena de formato:
%Y  año completo
%y  año sin siglo
%d  día del mes
%m  mes ('08')
%b  mes abreviado ('Aug')
%B  mes completo ('August')
%H  hora en formato 24 horas ('14')
%I  hora en formato 12 horas ('02')
%p 'AM' o 'PM'
%M  minuto ('30')
%S  segundo ('59')

### Validadores para rangos, conjuntos e igualdad
- IS_EQUAL_TO
- IS_NOT_EMPTY
- IS_NULL_OR: en desuso, en favor de IS_EMPTY_OR
- IS_EMPTY_OR: IS_EMPTY_OR(IS_DATE()), IS_EMPTY_OR(IS_ALPHANUMERIC(), null='anónimo')
- IS_EXPR: IS_EXPR(lambda v: T('no divisible por 3') if int(v) % 3 else None)
- IS_DECIMAL_IN_RANGE
- IS_FLOAT_IN_RANGE
- IS_INT_IN_RANGE
- IS_IN_SET

### Validadores de complejidad y seguridad
- IS_STRONG: IS_STRONG(min=10, special=2, upper=2)
- CRYPT

### Validadores especiales
- IS_LIST_OF
- IS_LIST_OF_EMAILS
- ANY_OF
- IS_IMAGE
- IS_UPLOAD_FILENAME
- IS_IPV4
- IS_IPV6
- IS_IPADDRESS

### Otros
- CLEANUP: Nunca falla, se usa para limpiar campos

### Validadores de bases de datos
- IS_NOT_IN_DB
- IS_IN_DB

## [SQLFORM.grid y SQLFORM.smartgrid](http://www.web2py.com/books/default/chapter/29/07/forms-and-validators#SQLFORM-grid-and-SQLFORM-smartgrid)
Atención: grid y smartgrid eran experimentales hasta la version 2.0, ya no lo son. Pero aún no se garantiza una compatibilidad de la capa de presentación en futuras versiones.

Son objetos que permiten crear controles CRUD, permitiendo paginación, navegación, búsqueda, ordenación, creación, actualización y borrado de registros.

Aprovechan SQLFORMs para la visualización, modificación y creación de registros, por lo que muchos de sus argumentos son precisamente para ser pasados a dichos formularios.

Dado que un grid atraviesa distintos estados, como editar un registro, este proceso generará una nueva petición. Dicha petición almacenará en request.args la información relativa al estado del grid.


## Clon de reddit (reddit)
Ahora que conocemos un poco mejor el funcionamiento de Web2py, vamos a crear un clon de reddit:
- Nuestra aplicación permitirá guardar links, organizados por categorías, con comentarios y votos.
- La idea es que los usuarios del sistema creen entradas (posts) relacionadas con alguna de las categorías permitidas. Dichas entradas contendrán un enlace a la fuente de dicha noticia o información.
- Estas entradas podrán ser votadas y/o comentadas por los usuarios del sistema.
- Dichos comentarios también podrán ser votados y comentados.

Nuestro modelo es más complejo que los que hemos creado hasta el momento. Incluirá, además de las tablas derivadas de la gestión de autorizaciones:
- Posts/entradas
- Votos a los posts
- Comentarios a los posts
- Votos a los comentarios

Voy a crear la nueva aplicación partiendo de una plantilla. En mi caso, voy a duplicar la aplicación blog2 y voy a eliminar lo que no necesito:
- Modifico .../reddit/views/layout.html para cambiar el nombre de la aplicación
- Borro el contenido de .../reddit/databases/
- Borro las vistas add.html, edit.html, index.html, manage.html y show.html del directorio ...reddit/views/default/
- Modifico el modelo .../reddit/db1.py y comento los modelos definidos
- Modifico el controlador .../reddit/default.py, para que incluya la acción user, que dejo sin tocar y la acción index que sera:
```python
# / -> página de inicio
def index():
    return dict(mensaje='Bienvenido a Mini Reddit')
```
- Creo la vista asociada:
```html
{{extend 'layout.html'}}
<h2>{{=mensaje}}</h2>
```

Con esto ya tenemos la aplicación lista para ir creciendo.

Primero voy a definir el modelo de mi aplicación. Podría crear un fichero para cada tabla, pero como en este caso son pocas tablas y con una estructura muy simple, voy a definir todas ellas en un único fichero
```python
# .../reddit/models/db1.py
# Categorías
#  - Nombre: SLUG en minúsculas
db.define_table('category',
                Field('name', requires=(IS_SLUG(),
                                        IS_LOWER(),
                                        IS_NOT_IN_DB(db, 'category.name'))))

# Posts o entradas
#  - Categoría a la que pertenece
#  - Título
#  - URL
#  - Cuerpo del post
#  - Votos recibidos
#  - Auth.signature
db.define_table('post',
                Field('category',
                      'reference category',
                      readable=False,
                      writable=False),
                Field('title', requires=IS_NOT_EMPTY()),
                Field('url', requires=IS_EMPTY_OR(IS_URL())),
                Field('body', 'text', requires=IS_NOT_EMPTY()),
                Field('votes', 'integer', default=0),
                auth.signature  # created_on/by, modified_on/by
                )

# Votos a los posts
#  - Post votado
#  - Votos acumulados: integer (cero por defecto)
#  - Auth.signature
db.define_table('vote',
                Field('post', 'reference post'),
                Field('score',
                      'integer',
                      default=0,
                      readable=False,
                      writable=False),
                auth.signature)

# Comentarios a los posts
#  - Post comentado
#  - Comentario comentado
#  - Votos acumulados del comentario
#  - Cuerpo del comentario
#  - Auth.signature
db.define_table('comm',
                 Field('post',
                       'reference post',
                       readable=False,
                       writable=False),
                 Field('parent_comm',
                       'reference comm',
                       readable=False,
                       writable=False),
                 Field('votes',
                       'integer',
                       default=0,
                       readable=False,
                       writable=False),
                 Field('body', 'text', requires=IS_NOT_EMPTY()),
                 auth.signature)

# Votos de los comentarios
#  - Comentario votado
#  - Votos acumulados
#  - Auth.signature
db.define_table('comm_vote',
                Field('comm', 'reference comm'),
                Field('score', 'integer', default=+1),
                auth.signature)
```

Voy a definir el API de mi aplicación, la forma en la que el usuario se relacionará con el sistema:
- .../reddit/default/index -> listado de categorías
- .../reddit/default/list_posts_by_votes/category_id/page_number -> listado de posts de una categoría ordenados por votos
- .../reddit/default/list_posts_by_datetime/category_id/page_number -> listado de posts de una categoría ordenados por fecha y hora
- .../reddit/default/create_post/category_id -> crea un post de una categoría
- .../reddit/default/view_post/post_id -> muestra un post en particular
- .../reddit/default/edit_post/post_id -> edita un post en particular
- .../reddit/default/list_posts_by_author/category_id/page_number -> muestra un listado de posts de un autor
- .../reddit/default/vote_post/post_id/up_down -> vota un post
- .../reddit/default/comm_vote_post/comm_id/up_down -> vota un comentario

Comenzaré con el listado de categorías:
<div class="card card-body bg-light" style="width:50%">
  <a class="btn btn_large btn-primary">Viajes</a>
</div>
<div class="card card-body bg-light" style="width:50%">
  <a class="btn btn_large btn-primary">Python</a>
</div>
<div class="card card-body bg-light" style="width:50%">
  <a class="btn btn_large btn-primary">Música</a>
</div>

Por el momento no hay categorías, de modo que debo crearlas antes de lanzar la aplicación. Esta será labor del administrador, ya que no quiero que los usuarios gestionen dicha tabla.

Cuando un usuario elija una categoría, debería ver una lista paginada con los posts relacionados. Dichos posts estarán ordenados por votos, pero vamos a permitir que el usuario los ordene por su momento de creación. Además dejaremos que cree un nuevo post para esa categoría.
<h3>Cocina</h3>
<button>Ordena por fecha y hora</button>
<button>Nuevo post</button>
<hr>
<div class="card card-body bg-light">
  <table>
    <tr data-id="post-1}">
      <td><button data-direction="down">-</button></td>
      <td><span class="votes">10</span></td>
      <td><button data-direction="up">+</button></td>
        <td><strong><a href="post.url">Título/URL del post 1</a></strong></td>
    </tr>
    <tr>
      <td colspan="3"></td>
      <td><a href="ver_post_id">Comentarios</a></td>
    </tr>
  </table>
  <br>
</div>
<div class="card card-body bg-light">
  <table>
    <tr data-id="post-1}">
      <td><button data-direction="down">-</button></td>
      <td><span class="votes">10</span></td>
      <td><button data-direction="up">+</button></td>
        <td><strong><a href="post.url">Título/URL del post ...</a></strong></td>
    </tr>
    <tr>
      <td colspan="3"></td>
      <td><a href="ver_post_id">Comentarios</a></td>
    </tr>
  </table>
  <br>
</div>
<div class="card card-body bg-light">
  <table>
    <tr data-id="post-1}">
      <td><button data-direction="down">-</button></td>
      <td><span class="votes">10</span></td>
      <td><button data-direction="up">+</button></td>
        <td><strong><a href="post.url">Título/URL del post n</a></strong></td>
    </tr>
    <tr>
      <td colspan="3"></td>
      <td><a href="ver_post_id">Comentarios</a></td>
    </tr>
  </table>
  <br>
</div>
<hr>
<button>Previa</button><button>Siguiente</button>

Cuando el usuario elija crear un post nuevo:
<h2>Di algo relativo a la categoría cocina</h2>
<span>Formulario del post</span><br>
<button>Enviar</button>

Cuando se acepte el post, el usuario lo verá:
<h2><a href="post.url">Título del post</a></h2>
<a href="edit.post.post_id">Edita tu post</a>
<div class="card card-body bg-light">
20/Febrero/2020 Ana dijo algo respecto a este post</div>
<div class="card card-body bg-light">
20/Febrero/2020 Luis dijo algo respecto a este post</div>
<div class="card card-body bg-light">
20/Febrero/2020 Javi dijo algo respecto a este post</div>
<span>Formulario para un nuevo comentario</span>

Cuando alguien vea un post, verá tambien sus comentarios y podrá comentarlo.

Además debería poder votar los propios comentarios. Es básicamente lo mismo que hemos hecho con los posts, pero aplicado a los comentarios.